Notebook found at https://github.com/senya-ashukha/real-nvp-pytorch but with additional comments as I read the Real NVP paper (https://arxiv.org/pdf/1605.08803.pdf) and learn PyTorch syntax. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
rcParams['figure.dpi'] = 300

import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter

from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler

In [2]:
class RealNVP(nn.Module): # base class Module 
    def __init__(self, nets, nett, mask, prior):
        super(RealNVP, self).__init__()
        
        self.prior = prior
        self.mask = nn.Parameter(mask, requires_grad=False)
        self.t = torch.nn.ModuleList([nett() for _ in range(len(masks))]) # translation function (net) 
        self.s = torch.nn.ModuleList([nets() for _ in range(len(masks))]) # scaling function (net)
        # nn.ModuleList is just like a Python list. It was designed to store any desired number of nn.Module’s.
        
    def g(self, z):
        x = z
        for i in range(len(self.t)): # for each layer 
            x_ = x*self.mask[i] # splitting features between channels. 
                                # features selected here used to compute s(x) and f(x) but not updated themselves yet.  
            s = self.s[i](x_)*(1 - self.mask[i]) 
            t = self.t[i](x_)*(1 - self.mask[i])
            x = x_ + (1 - self.mask[i]) * (x * torch.exp(s) + t)
        return x

    def f(self, x):
        log_det_J, z = x.new_zeros(x.shape[0]), x
        # new_zeros(size) returns a Tensor of size size filled with 0 
        for i in reversed(range(len(self.t))): # move backwards through layers 
            z_ = self.mask[i] * z # tensor of size num samples x num features
            s = self.s[i](z_) * (1-self.mask[i]) # self.s[i] is the entire sequence of scaling operations
            t = self.t[i](z_) * (1-self.mask[i])
            z = (1 - self.mask[i]) * (z - t) * torch.exp(-s) + z_
            log_det_J -= s.sum(dim=1)
            # each pass through here applies all operations defined in nets() and all the ones defined in nett() 
        # self.s[1](z_) is not the same as self.s[3](z_)
        return z, log_det_J
    
    def log_prob(self,x):
        z, logp = self.f(x) # z = f(x) 
        return self.prior.log_prob(z) + logp
        
    def sample(self, batchSize): 
        z = self.prior.sample((batchSize, 1))
        logp = self.prior.log_prob(z)
        x = self.g(z)
        return x

In [3]:
# create net s and net t 
nets = lambda: nn.Sequential(nn.Linear(2, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 2), nn.Tanh())
nett = lambda: nn.Sequential(nn.Linear(2, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 2)) 
#    nn.Linear(256, 256) means 256 input features, 256 output features 

masks = torch.from_numpy(np.array([[0, 1], [1, 0]] * 3).astype(np.float32)) # 6x2 matrix. len(masks) = 6 = num subblocks. 
# so we have a total of 3 neural blocks (see fig. 1 of boltzmann generators paper)
prior = distributions.MultivariateNormal(torch.zeros(2), torch.eye(2))
flow = RealNVP(nets, nett, masks, prior) # define our model 

In [ ]:
# torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, 
#    amsgrad=False): Implements Adam algorithm.

optimizer = torch.optim.Adam([p for p in flow.parameters() if p.requires_grad==True], lr=1e-4)
for t in range(5000):    
    noisy_moons = datasets.make_moons(n_samples=100, noise=.05)[0].astype(np.float32) # Make two interleaving half circles
    loss = -flow.log_prob(torch.from_numpy(noisy_moons)).mean()
    
    optimizer.zero_grad() # we need to set the gradients to zero before starting to do 
                          # backpropragation because PyTorch accumulates the gradients on 
                          # subsequent backward passes.
    loss.backward(retain_graph=True)
    optimizer.step()
    
    if t % 500 == 0:
        print('iter %s:' % t, 'loss = %.3f' % loss)

iter 0: loss = 2.478
iter 500: loss = 0.520


In [ ]:
# datasets.make_moons(): Make two interleaving half circles
# torch.from_numpy(ndarray) → Tensor: Creates a Tensor from a numpy.ndarray

# plot data points in latent space
noisy_moons = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32)
# index [0] to skip returning the class labels Y 
z = flow.f(torch.from_numpy(noisy_moons))[0].detach().numpy()
plt.subplot(221)
plt.scatter(z[:, 0], z[:, 1])
plt.title(r'$z = f(X)$')

# sample from latent distribution 
z = np.random.multivariate_normal(np.zeros(2), np.eye(2), 1000)
plt.subplot(222)
plt.scatter(z[:, 0], z[:, 1])
plt.title(r'$z \sim p(z)$')

# plot actual data
plt.subplot(223) 
x = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32)
plt.scatter(x[:, 0], x[:, 1], c='r')
plt.title(r'$X \sim p(X)$')

# plot transformation of the points sampled in latent space
plt.subplot(224) 
x = flow.sample(1000).detach().numpy() 
plt.scatter(x[:, 0, 0], x[:, 0, 1], c='r')
plt.title(r'$X = g(z)$')

# use of tensor.detach(): https://discuss.pytorch.org/t/clone-and-detach-in-v0-4-0/16861/2